In [1]:
import jax.numpy as jnp
import numpy as np
import time
import chex
import jax
import gymnasium as gym
import exciting_environments as excenv
import random
import matplotlib.pyplot as plt
import diffrax

### Simulate Ahead

In [5]:
env=excenv.CartPole(batch_size=5)

In [6]:
acts=jnp.full((5,8),jnp.array([1,1,1,1,1,1,1,1])*10)[:,:,None]

In [7]:
states=env.init_state()
obs,rew,truncated,terminated,last_state=env.vmap_sim_ahead(states,
                   acts,
                   env.tau,
                   env.tau
                   )
obs[0]

Array([[ 0.        ,  0.        , -1.        ,  0.        ],
       [ 0.        ,  0.48780486, -1.        ,  0.7317073 ],
       [ 0.03252032,  0.97560847, -0.96273446,  1.4634118 ],
       [ 0.09756088,  1.4607671 , -0.8882036 ,  2.1818695 ],
       [ 0.19494535,  1.9311341 , -0.7770819 ,  2.8316896 ],
       [ 0.3236876 ,  2.3645494 , -0.632865  ,  3.3050861 ],
       [ 0.48132423,  2.7501717 , -0.4645385 ,  3.5059454 ],
       [ 0.66466904,  3.1163695 , -0.2859819 ,  3.4083133 ],
       [ 0.872427  ,  3.5162072 , -0.11239797,  3.0059807 ]],      dtype=float32, weak_type=True)

In [8]:
state=env.init_state()
for _ in range(acts.shape[1]):
    obs,state=env.vmap_step(action=10*jnp.ones(5).reshape(-1,1),state=state)
    print(obs[0,:])

[ 0.          0.48780486 -1.          0.7317073 ]
[ 0.03252032  0.97560847 -0.96273446  1.4634118 ]
[ 0.09756088  1.4607671  -0.88820356  2.1818695 ]
[ 0.19494535  1.9311341  -0.77708185  2.8316898 ]
[ 0.32368764  2.3645494  -0.632865    3.3050866 ]
[ 0.48132423  2.7501714  -0.46453837  3.505946  ]
[ 0.66466904  3.1163695  -0.28598207  3.4083138 ]
[ 0.872427    3.5162072  -0.11239804  3.0059812 ]
